<a href="https://colab.research.google.com/github/yukinaga/langchain/blob/main/section_2/01_lcel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain Expression Language

LCEL（LangChain Expression Language）を使うことで、基本的なコンポーネントから複雑なチェーンを簡単に構築することができます。  
今回は、「Prompt」「Model」「Output parser」をLCELでつなぎます。  
なお、以下の公式ドキュメントを参考にしています。  
https://python.langchain.com/docs/


## ライブラリのインストール
langchain-coreとlangchain-openaiをインストールします。  

In [ ]:
!pip install langchain-core langchain-openai

## API Keyの設定
OpneAIのAPIを使用するために必要な「API key」を設定します。  
以下のコードの、  
`os.environ["OPENAI_API_KEY"] = "Your API key"`  
における  
`Your API key`の箇所を、自分のAPI keyに置き換えます。  
OpenAIのAPI keyは、OpenAIのサイトで取得できます。   
https://platform.openai.com/account/api-keys


In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "Your API key"

API keyの流出にはリスクがあります。  
他者に知られないように、慎重に扱ってください。

## LCELの例
以下は「Prompt」 、「Model」、そして「Output parser」をLCELによる組み合わせた例です。


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("{topic}に関するなぞなぞを考えてください。")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser  # LCELで複数のコンポーネントを1つにまとめる

chain.invoke({"topic": "柴犬"})

このコードでは、LCELを使って異なるコンポーネントを1つのチェーンにまとめています。  
  
「｜」は、異なるコンポーネントを連結し、あるコンポーネントからの出力を次のコンポーネントへの入力として渡します。  

## Prompt
Promptは辞書を取り込んで PromptValue を生成します。  
PromptValueはプロンプトのラッパーで、Modelに渡すことができます。  

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("私の{topic}を占ってください。")
prompt_value = prompt.invoke({"topic": "金運"})
prompt_value

## Model
PromptValueはModel に渡されます。  
ModelにはLLMとChatModelがあります。  
LLMは文章が入力で、ChatModelは文章のやりとりが入力です。  
以下ではChatModelを使っていますが、ChatModelであればBaseMessageを出力します。  

In [ ]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-3.5-turbo")
message = chat_model.invoke(prompt_value)
message

ModelがLLMであれば、文字列を出力します。  

In [ ]:
from langchain_openai.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo-instruct")
message = llm.invoke(prompt_value)
message

## Output parser
Output parserは、文字列かBaseMessageを入力として受け取ります。  
以下では、StrOutputParserを使っていますが、これは単純に入力を文字列に変換します。

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(message)

コードの実行後は、OpenAIのサイトでAPIの使用量を確認しましょう。  
https://platform.openai.com/account/usage